In [13]:
%load_ext autoreload 
%autoreload 2

from p1_graphs import *
import altair as alt
data = get_accident_data(fname="dataset_v1.csv",sample=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
select_bar = alt.selection_point(encodings=['x'], empty='all')
# make a bar chart of number of accidents by month, use altair transform_timeunit to extract month from date
chart1=alt.Chart(data).mark_point().encode(
    x=alt.X('month(date):T', title='Month'),
    y=alt.Y('count()', title='Number of Accidents'),
    color=alt.condition(select_bar, 'Severity', alt.value('lightgray'))
).add_params(select_bar)

# make scatter plot of accidents 

c:\Users\pamar\anaconda3\envs\VI\Lib\site-packages\altair\vegalite\v5\api.py:394: AltairDeprecationWarning: The value of 'empty' should be True or False.
  warnings.warn(


In [65]:
# make scatter plot of accidents by month
chart2=alt.Chart(data).mark_circle().encode(
    x=alt.X('month(date):T', title='Month'),
    y=alt.Y('count()', title='Number of Accidents'),
    color=alt.condition(select_bar, alt.value('red'), alt.value('lightgray'))
).add_selection(select_bar).transform_filter(select_bar)
chart3 = alt.Chart(data).mark_point().encode(
    x=alt.X('month(date):T', title='Month'),
    y=alt.Y('count()', title='Number of Accidents'),
    color=alt.condition(select_bar, alt.value("green"), alt.value('lightgray'))
).add_params(select_bar)

chart1 | chart2 | chart3

c:\Users\pamar\anaconda3\envs\VI\Lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.HConcatChart(...)

In [21]:
width = 300
height = 200
morning_rh = {}
morning_rh["x1"] = "08:00"
morning_rh["x2"] = "09:00"
morning_rh = pd.DataFrame([morning_rh])

afternoon_rh = {}
afternoon_rh["x1"] = "15:00"
afternoon_rh["x2"] = "19:00"
afternoon_rh = pd.DataFrame([afternoon_rh])

morning_rh["x1"] = pd.to_datetime(morning_rh["x1"])
morning_rh["x2"] = pd.to_datetime(morning_rh["x2"])

afternoon_rh["x1"] = pd.to_datetime(afternoon_rh["x1"])
afternoon_rh["x2"] = pd.to_datetime(afternoon_rh["x2"])

morning_window = (
    alt.Chart(morning_rh)
    .mark_rect(opacity=0.1)
    .encode(x="hours(x1):T", x2="hours(x2):T", color=alt.value("gray"))
)

afternoon_window = (
    alt.Chart(afternoon_rh)
    .mark_rect(opacity=0.1)
    .encode(x="hours(x1):T", x2="hours(x2):T", color=alt.value("gray"))
)

before_after = (
    alt.Chart(width=width, height=height)
    .mark_area(size=3, opacity=0.4, interpolate="basis")
    .encode(
        x=alt.X("hours(HOUR):T"),
        y=alt.Y("count()").stack(None, title=None),
        color=alt.Color(
            "covid:N", scale=alt.Scale(range=[colors["col1"], colors["col2"]])
        ),
    )
)
data["HOUR"] = pd.to_datetime(data["CRASH TIME"])

chart = (
    alt.layer(before_after, morning_window, afternoon_window, data=data).transform_filter(select_bar)
    .facet(row=alt.Row("weekday", title=None, header=alt.Header(labelFontSize=16)))
    .configure_axis(title=None).add_params(select_bar)
)

In [25]:
chart1

alt.Chart(...)